# CSS5228 Final Project

In this notebook, we will start training our model

- Random Forest Base - MAE: 331763.892261509

In [152]:
import pandas as pd
import numpy as np
from pathlib import Path

from efficient_apriori import apriori

import matplotlib.pyplot as plt
import seaborn as sns


In [101]:
df_train = pd.read_csv("../clean_data/train_preproc.csv", index_col=0)
df_test = pd.read_csv("../clean_data/test_preproc.csv", index_col=0)

In [154]:
sns.set_theme(style="ticks")
sns.set_context("paper", rc={"xtick.labelsize":15,
                             "ytick.labelsize":15,
                             "axes.titlesize":20,
                             "axes.labelsize":15})
default_col = '#4c72b0'
highlight_col = '#c44d52'

img_dir = Path("../images/")